References:
https://www.kaggle.com/slashtea/domain-generation-algorithm?select=words.txt
https://splunkbase.splunk.com/app/3559/#/details
Cisco Database: http://s3-us-west-1.amazonaws.com/umbrella-static/index.html

In [2]:
import pandas as pd
import numpy as np

1. Data Exploration

Reading and cleaning the data into teh usable format

In [3]:
words_loc = "archive/words.txt"
top_1m_loc = "archive/top-1m.csv"
dga_project_top_1m_loc = "archive/dga_project_top-1m.csv"
dga_project_dga_domain_list_clean_loc = "archive/dga_project_dga_domain_list_clean.txt"

In [4]:
top_1m = pd.read_csv(top_1m_loc)
dga_project_top_1m = pd.read_csv(dga_project_top_1m_loc)
top_1m.head()

,1,google.com
0,2,youtube.com
1,3,tmall.com
2,4,qq.com
3,5,baidu.com
4,6,sohu.com


In [5]:
dga_project_top_1m.head()

,1,google.com
0,2,youtube.com
1,3,tmall.com
2,4,baidu.com
3,5,qq.com
4,6,sohu.com


In [6]:
words = open(words_loc, "r")
words = words.read()
words = words.split()

dga_project_dga_domain_list_clean = open(dga_project_dga_domain_list_clean_loc, "r")
dga_project_dga_domain_list_clean = dga_project_dga_domain_list_clean.read()
dga_project_dga_domain_list_clean = dga_project_dga_domain_list_clean.split()

In [7]:
print(words[:5])
print(dga_project_dga_domain_list_clean[:10])

['2', '1080', '&c', '10-point', '10th']
['nymaim', 'vvqbhhwma.org', '2021-05-11', '00:00:00', '2021-05-11', '23:59:59', 'nymaim', 'cfhauqbaz.com', '2021-05-11', '00:00:00']


In [8]:
dga_domain = dga_project_dga_domain_list_clean[1::6]
print(dga_domain[:5])

dga_domain_subclasses = dga_project_dga_domain_list_clean[::6]
print(dga_domain_subclasses[:5])

['vvqbhhwma.org', 'cfhauqbaz.com', 'bvxjsbkqu.biz', 'hggazskvkdy.com', 'xgoqyxgfgm.com']
['nymaim', 'nymaim', 'nymaim', 'nymaim', 'nymaim']


Creating a dataset with labels.
We will combine "top_1m" with label 0 (not dga) and "dga_project_dga_domain_list_clean_domains" with the label 1 (dga).
Two rows: name, label

Combining feature matrix and Labels matrix

In [9]:
data[0][:] = 

SyntaxError: invalid syntax (Temp/ipykernel_12140/2992545012.py, line 1)

2. Feature Engineering

3. Create Machine Learnings Models

4. Operationalize ML

5. Test and Benchmark